In [2]:
import os
os.cpu_count()

8

In [27]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from scipy.stats import uniform, randint

In [4]:
titanic = pd.read_csv('train.csv')

In [5]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
titanic.drop(columns = 'Cabin', inplace = True)

In [7]:
titanic['Age'].fillna(titanic['Age'].mean(), inplace = True)

In [8]:
titanic['Embarked'].fillna(titanic['Embarked'].mode()[0], inplace = True)

In [9]:
label_encoder = LabelEncoder()
titanic['Embarked'] = label_encoder.fit_transform(titanic['Embarked'])

In [10]:
titanic['Sex'] = label_encoder.fit_transform(titanic['Sex'])

In [11]:
X = titanic.drop(columns= ['Name', 'Ticket', 'Fare','Survived'])
y = titanic['Survived']

In [12]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [17]:
svc =SVC()

rand_param = {
    'C' : uniform(0.1, 100),
    'kernel' : [ 'linear', 'rbf'],
    'gamma' : ['scale'],
}

In [18]:
random_search = RandomizedSearchCV(estimator = svc,
                                   param_distributions = rand_param,
                                  n_iter = 10,
                                  cv = 5,
                                  scoring = 'accuracy',
                                  random_state = 1,
                                  verbose = 2,
                                  n_jobs = -1)

random_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000026AB922DA10>,
                                        'gamma': ['scale'],
                                        'kernel': ['linear', 'rbf']},
                   random_state=1, scoring='accuracy', verbose=2)

In [19]:
y_pred = random_search.predict(X_test)

In [20]:
ACC = accuracy_score(y_pred, y_test)
ACC

0.8097014925373134

In [41]:
model_params = {
   
    'random_forest' : {
    'model': RandomForestClassifier(),
    'params':{
        'n_estimators': randint(10,100),
        'max_depth' : randint(1,20),
        'min_samples_split': randint(2, 20),
        'min_samples_leaf' : randint(2,20)
        }
    },

    'logistic_regression' : {
        'model' : LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            'C': uniform(1,10),
            'penalty' : ['l1', 'l2']
        }
    }
}

    

In [42]:
for model_name, mp in model_params.items():
    print(f'Running randomised Search For  : {model_name}')

    Random_Search = RandomizedSearchCV( mp['model'], mp['params'], cv = 3, n_jobs= -1, n_iter = 10)  
    Random_Search.fit(X_train, y_train)
    
    print(f'The Best Parameters for : {model_name} : {Random_Search.best_params_}')
    print(f'The Best Score For: {model_name} : {Random_Search.best_score_}')

Running randomised Search For  : random_forest
The Best Parameters for : random_forest : {'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 13, 'n_estimators': 87}
The Best Score For: random_forest : 0.8186160658986746
Running randomised Search For  : logistic_regression
The Best Parameters for : logistic_regression : {'C': 3.2360858476195666, 'penalty': 'l2'}
The Best Score For: logistic_regression : 0.7993543292456335
